# Example classification with MixedInputModel

In [1]:
import pandas as pd
import numpy as np
from utils import *
from model import *

# saved bulldozers data from the first lessons
dep = 'SalePrice'
PATH = "data/bulldozers/"
df_raw = pd.read_feather('tmp/bulldozers-raw')
keep_cols = list(np.load('tmp/keep_cols.npy'))

df_raw.loc[df_raw.YearMade<1950, 'YearMade'] = 1950
df_raw['age'] = df_raw.saleYear-df_raw.YearMade
df_raw = df_raw[keep_cols+['age', dep]].copy()
df_indep = df_raw.drop(dep,axis=1)

n_valid = 12000
n_trn = len(df_raw)-n_valid


cat_flds = [n for n in df_indep.columns if df_raw[n].nunique()<n_trn/50]
' '.join(cat_flds)

for o in ['saleElapsed', 'saleDayofyear', 'saleDay', 'age', 'YearMade']: cat_flds.remove(o)
[n for n in df_indep.drop(cat_flds,axis=1).columns if not is_numeric_dtype(df_raw[n])]


for n in cat_flds: df_raw[n] = df_raw[n].astype('category').cat.as_ordered()

cont_flds = [n for n in df_indep.columns if n not in cat_flds]

df_raw = df_raw[cat_flds+cont_flds+[dep]]
df, y, nas, mapper = preprocessing(df_raw, 'SalePrice', do_scale=True)

val_idx = list(range(n_trn, len(df)))

emb_c = {n: len(c.cat.categories)+1 for n,c in df_raw[cat_flds].items()}

emb_szs = [(c, min(50, (c+1)//2)) for _,c in emb_c.items()]

In [2]:
# turn `SalePrice` into a categorical variable for a classification problem 
categorical_y = []
for i in y:
    if i < 9.0:
        categorical_y.append('low')
    elif i > 11.0:
        categorical_y.append('high')
    else:
        categorical_y.append('mid')


from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
categorical_y = le.fit_transform(categorical_y)
categorical_y = np.asarray(categorical_y)
print(categorical_y)

[0 2 2 ... 2 2 1]


In [3]:
from fastai.column_data import ColumnarModelData, StructuredModel
import torch.nn as nn
import torch
from fastai.learner import *
import torch.nn.functional as F

PATH = 'data/'
md = ColumnarModelData.from_data_frame(PATH, val_idx, df, categorical_y, cat_flds=cat_flds, bs=32)

In [4]:
#Create own MixedInputModel class and StructuredLearner class 

def emb_init(x):
    x = x.weight.data
    sc = 2/(x.size(1)+1)
    x.uniform_(-sc,sc)

    
class MixedInputModelClassificaiton(nn.Module):
    def __init__(self, emb_szs, n_cont, emb_drop, out_sz, szs, drops,
                 y_range=None, use_bn=False):
        super().__init__()
        self.embs = nn.ModuleList([nn.Embedding(c, s) for c,s in emb_szs])
        for emb in self.embs: emb_init(emb)
        n_emb = sum(e.embedding_dim for e in self.embs)
        self.n_emb, self.n_cont=n_emb, n_cont
        
        szs = [n_emb+n_cont] + szs
        self.lins = nn.ModuleList([
            nn.Linear(szs[i], szs[i+1]) for i in range(len(szs)-1)])
        self.bns = nn.ModuleList([
            nn.BatchNorm1d(sz) for sz in szs[1:]])
        for o in self.lins: kaiming_normal(o.weight.data)
        self.outp = nn.Linear(szs[-1], out_sz)
        kaiming_normal(self.outp.weight.data)

        self.emb_drop = nn.Dropout(emb_drop)
        self.drops = nn.ModuleList([nn.Dropout(drop) for drop in drops])
        self.bn = nn.BatchNorm1d(n_cont)
        self.use_bn,self.y_range = use_bn,y_range

    def forward(self, x_cat, x_cont):
        if self.n_emb != 0:
            x = [e(x_cat[:,i]) for i,e in enumerate(self.embs)]
            x = torch.cat(x, 1)
            x = self.emb_drop(x)
        if self.n_cont != 0:
            x2 = self.bn(x_cont)
            x = torch.cat([x, x2], 1) if self.n_emb != 0 else x2
        for l,d,b in zip(self.lins, self.drops, self.bns):
            x = F.relu(l(x))
            if self.use_bn: x = b(x)
            x = d(x)
        x = self.outp(x)
        # change to softmax output
        x = F.log_softmax(x, dim=-1)
        return x
    
    
class StructuredLearner(Learner):
    def __init__(self, data, models, **kwargs):
        super().__init__(data, models, **kwargs)
        # change to a classification loss function, 
        # might not need this if you define your own training loop
        self.crit = nn.NLLLoss()

In [5]:
# change to total number of classes 
num_classes = len(set(categorical_y))
model = MixedInputModelClassificaiton(emb_szs, len(cont_flds), 0.04, num_classes, [1000,500], [0.001,0.01])
m = StructuredLearner(md, StructuredModel(model))

In [6]:
# pass in your categorical features, and continious features into your model()
# example 
cat, cont, y = next(iter(md.trn_dl))
cat, cont, y = Variable(cat), Variable(cont), Variable(y).long()
pred = model(cat, cont)
for p, true in zip(pred.data.numpy(), torch.max(y, 1)[0].data):
    print('pred log probs: {} -- True: {}'.format(p, true))

pred log probs: [-1.04458 -1.10577 -1.14822] -- True: 2
pred log probs: [-0.98385 -1.07938 -1.25063] -- True: 2
pred log probs: [-1.00442 -1.16467 -1.13408] -- True: 2
pred log probs: [-1.12615 -1.03527 -1.13759] -- True: 2
pred log probs: [-0.9649  -1.12864 -1.21906] -- True: 2
pred log probs: [-1.20009 -1.03089 -1.07253] -- True: 2
pred log probs: [-1.01543 -1.14942 -1.13652] -- True: 2
pred log probs: [-1.03087 -1.1107  -1.15845] -- True: 0
pred log probs: [-1.06692 -1.13093 -1.09901] -- True: 2
pred log probs: [-1.16797 -1.10318 -1.02949] -- True: 0
pred log probs: [-1.0332  -1.08289 -1.18574] -- True: 0
pred log probs: [-1.06535 -0.98435 -1.26688] -- True: 2
pred log probs: [-1.26336 -1.03621 -1.01472] -- True: 2
pred log probs: [-1.06225 -1.01288 -1.23391] -- True: 2
pred log probs: [-1.14893 -1.03823 -1.11187] -- True: 0
pred log probs: [-1.25545 -1.02609 -1.03101] -- True: 2
pred log probs: [-1.05491 -1.0416  -1.20769] -- True: 1
pred log probs: [-1.13548 -1.05051 -1.11178] -- 